In [1]:
# This tutorial is meant as an introduction to some of BrainEx's important parameters. Knowing these parameters
# will allow you greater control over the functions of BrainEx. 
from brainex.utils.gxe_utils import from_csv, from_db

In [2]:
# First, let's discuss the from_csv and from_db functions. 
# These functions (as you may recall from the saving + loading tutorial) are used to create a BrainEx Engine object. from_csv
# creates one based off of a datafile, and from_db loads a previously saved BrainEx Engine. Both from_csv and from_db have 
# parameters that allow you to define the resources given to your BrainEx Engine. 

# from_csv

data = './ItalyPower.csv' # The first parameter of from_csv is data. This is a path to the data file you wish to use. 

feature_num = 0 # Feature num is the number of columns before the timeseries data begins in your data file. Some data files 
# have data stored before the timeseries itself begins on each row. If your dataset has these columns then feature_num must
# be equal to the number of these columns. The dataset we're testing on doesn't have any so feature_num is 0.

use_spark = False # If this bool is set to True then your BrainEx Engine will use Spark. If it's set to False it won't. 

header = 0 # This is the number of header rows before the data in your dataset. Header rows usually name the different columns
# and most datasets either have one or none. Our dataset this time doesn't have any. 

driver_mem = 1 # This parameter defines the amount of memory (RAM) given to the main BrainEx process. In effect, 
# this is the number of gigabytes of RAM that BrainEx will run with. This number is limited by the amount of RAM available
# on your computer. For larger datasets I would suggest using more RAM, but our test dataset is small so 1 gb should be fine.

max_result_mem = 1 # This parameter defines the amount of memory (RAM) given to make the result. As with driver_mem, 
# larger datasets will require more RAM. If you are unsure how much to use I would suggest giving driver_mem and max_result_mem
# half of your memory each.

_rows_to_consider = None # By default BrainEx uses all rows of the inputted dataset, but if you would like to limit the number
# of rows that the system will search through then you can use this parameter. If _rows_to_consider is not included in the 
# from_csv invocation then the whole dataset will be searched through. If in doubt I would suggest not using this parameter. 

num_worker = 1 # num_worker sets the number of cores that BrainEx will use to preprocess the dataset. The larger the number of
# cores the faster your dataset will be processed, but you are limited by the number of cores in your computer. num_worker must
# be at least 1.

# Now, let's use the parameters we set above. 
brainexengine = from_csv(data, feature_num = feature_num, use_spark = use_spark, header = header, driver_mem = driver_mem, max_result_mem = max_result_mem, num_worker = num_worker)

msg: from_csv, feature num is 0, auto-generating uuid
Not using z-normalization
 Genex Engine: Using Python Native Multiprocessing


In [3]:
# Now that we have a working BrainEx Engine object let's try saving it so we can use it for from_db. 
brainexengine.save('~/savelocation')

In [6]:
# from_db

path = '~/savelocation' # Path, the first parameter of from_db, points to the saved BrainEx Engine. In this case, we just saved
# our engine to '~/savelocation' so we should set path to the same value. 

num_worker = 1 # Like in from_csv, num_worker is the number of cores given to BrainEx. It must be at least 1, and can't be 
# higher than the number of cores available in your computer. 

driver_mem = 2 # Again, driver_mem is the same here as in from_csv. 

max_result_mem = 2 # max_result_mem is the same here as in from_csv. 

bxe = from_db(path = path, num_worker = num_worker, driver_mem = driver_mem, max_result_mem = max_result_mem)

 Genex Engine: Using Python Native Multiprocessing


In [7]:
# Now that we've explored the functions that create BrainEx Engines, let's look at their functions of the engines themselves. 
# The first is the build function.

# brainexengine.build

st = 0.1 # st stands for similarity threshold. The similarity threshold can be between 0 and 1 (but not 0 or 1) and it controls
# how the clusters are built during preprocessing. Larger values result in larger clusters that are few in number. On the other
# hand, smaller values result in more, smaller clusters. There is no one correct value, but 0.1 is often a good starting place.

dist_type = 'eu' # The distance type controls the distance metric used in clustering. Preprocessing with different distance
# metrics can result in different results. The available options are 'eu' (Euclidean), 'ma' (Manhattan), and 'ch' (Chebyshev).

loi = None # loi stands for length of interest. If you are only interested in subsections of a certain length, then you can use
# loi to preprocess only subsequences of that length. However,if you do so, you will only be able to query subsequences with the
# same lenght as the loi. If set to None, BrainEx will preprocess subsequences of every length. I would suggest not using this 
# parameter unless you have a specific goal in mind. 

verbose = 1 # The parameter just controls whether or not BrainEx will print information about the build process as it goes.
# If set to 1 it will print this info, and if set to 0 it will not. 

brainexengine.build(st = st, dist_type = dist_type, loi = loi, verbose = verbose)

{'self': <brainex.database.BrainexEngine.BrainexEngine object at 0x7f3c024b9f90>, 'st': 0.1, 'dist_type': 'eu', 'loi': None, 'verbose': 1, '_group_only': False, '_use_dss': True, '_use_dynamic': False}


In [11]:
# Finally, let's examine the important parameters used in querying.

# brainexengine.query

query = brainexengine.get_random_seq_of_len(15, seed=1) # Query is the subsequence of the dataset that you want BrainEx to match
# against. In this case I am using the get_random_seq_of_len function to set the query to a random subsequence of our dataset.

best_k = 5 # best_k is used to define the number of matches you wish BrainEx to find. It must be at least one. 

loi = None # loi stands for length of interest, and, like in the build function, it controls the length of the sequences that
# BrainEx searches through. If you have a specific length you want to look for then you can use this parameter, but otherwise
# leaving it as None causes BrainEx to search through all lengths in the dataset.

exclude_same_id = False # This bool controls whether or not the query sequence will be excluded from the results. By default it
# is set to False which generally causes the query sequence to be included in the results. 

overlap = 1.0 # Overlap is a float that must be between 0 and 1 (inclusive). If overlap is used, then the results returned by
# the query function will overlap at most overlap%. If your results are too similar you can use this parameter to ensure that
# only results that are dissimilar to each other will be included. 

brainexengine.query(query = query, best_k = best_k, loi = loi, exclude_same_id = exclude_same_id, overlap = overlap)

[(0.0, <brainex.classes.Sequence.Sequence at 0x7f3c00b02d10>),
 (0.0765400441427873, <brainex.classes.Sequence.Sequence at 0x7f3c00ac0ad0>),
 (0.08013465954685774, <brainex.classes.Sequence.Sequence at 0x7f3c02653690>),
 (0.09014727756125093, <brainex.classes.Sequence.Sequence at 0x7f3c02653250>),
 (0.09055574294203451, <brainex.classes.Sequence.Sequence at 0x7f3c026532d0>)]